PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2MB 73kB/s 
     |████████████████████████████████| 204kB 52.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=0e9b60df0f0105a215b6d33bf522428487cb8f9511ffb7b5d954afddfcc0c016
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

# 보스턴 주택 가격 예측 모델 만들기




In [3]:
spark

데이터 다운로드

In [4]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2021-02-04 11:28:53--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.236.81
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.236.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.08s   

2021-02-04 11:28:54 (434 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [5]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Feb  1 17:27 sample_data
-rw-r--r-- 1 root root 36240 Jan 31 01:46 boston_housing.csv


데이터 읽기

In [6]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [7]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [8]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [9]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

예측하고자 하는 것은 label field이다. label field를 제외하고 피쳐들을 묶어서 하나의 벡터로 만들어준다. 

In [10]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [11]:
data_2 = assembler.transform(data) # transform은 하나의 데이터프레임을 받아서 새로운 column을 만들어 주는 함수

In [12]:
data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [13]:
train, test = data_2.randomSplit([0.7, 0.3])

In [14]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

## 모델 성능 측정

In [15]:
evaluation_summary = model.evaluate(test)

In [16]:
evaluation_summary

In [17]:
evaluation_summary.meanAbsoluteError

3.3652981526964014

In [18]:
evaluation_summary.rootMeanSquaredError

5.02230708317449

In [19]:
evaluation_summary.r2

0.6484273399626419

## 모델 예측값 살펴보기 

In [20]:
predictions = model.transform(test) #test셋 데이터에다가 prediction결과 column을 붙여서 반환

In [21]:
predictions.show()

+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096|55.0| 2.25|   0| 0.389|6.453|31.9| 7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|27.392984479558002|
|0.01501|80.0| 2.01|   0| 0.435|6.635|29.7|  8.344|  4|280|   17.0|390.94| 5.99|24.5|[0.01501,80.0,2.0...|27.470800186707457|
|0.01709|90.0| 2.02|   0|  0.41|6.728|36.1|12.1265|  5|187|   17.0|384.46|  4.5|30.1|[0.01709,90.0,2.0...|24.990054046942745|
|0.01951|17.5| 1.38|   0|0.4161|7.104|59.5| 9.2229|  3|216|   18.6|393.24| 8.05|33.0|[0.01951,17.5,1.3...|23.381070211006588|
|0.02177|82.5| 2.03|   0| 0.415| 7.61|15.7|   6.27|  2|348|   14.7|395.38| 3.11|42.3|[0.02177,82.5,2.0...| 38.08237176

In [22]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|22.0|[0.01096,55.0,2.2...|27.392984479558002|
|24.5|[0.01501,80.0,2.0...|27.470800186707457|
|30.1|[0.01709,90.0,2.0...|24.990054046942745|
|33.0|[0.01951,17.5,1.3...|23.381070211006588|
|42.3|[0.02177,82.5,2.0...| 38.08237176185598|
|31.1|[0.02187,60.0,2.9...| 32.60889216793642|
|34.7|[0.02729,0.0,7.07...|30.818239773354968|
|26.6|[0.02899,40.0,1.2...| 22.33678631025488|
|28.7|[0.02985,0.0,2.18...| 24.58931223955104|
|31.2|[0.03049,55.0,3.7...| 28.78689277172363|
|17.5|[0.03113,0.0,4.39...|16.121570724292795|
|33.4|[0.03237,0.0,2.18...|28.485423171812148|
|19.5|[0.03427,0.0,5.19...|  19.5991542518526|
|35.4|[0.03705,20.0,3.3...| 34.45818915079085|
|23.2|[0.03871,52.5,5.3...|26.761603973176328|
|28.0|[0.04113,25.0,4.8...| 28.54909059169519|
|22.9|[0.04203,28.0,15....| 29.31688706487535|
|20.6|[0.04294,28.0,15....|27.087043482013165|
|20.5|[0.0433

모델 저장하기

In [ ]:
model.save("boston_housing_model")

In [26]:
!ls boston_housing_model

data  metadata


In [27]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Feb  4 11:43 data
drwxr-xr-x 2 root root 4096 Feb  4 11:43 metadata


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model2" 
model.save(path)

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [ ]:
predictions2 = loaded_model.transform(test)

In [ ]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|22.0|[0.01096,55.0,2.2...|26.571482297719307|
|32.7|[0.01301,35.0,1.5...|30.167506151964368|
|35.4|[0.01311,90.0,1.2...|30.335900587302184|
|18.9|[0.0136,75.0,4.0,...|15.228505739892949|
|50.0|[0.01501,90.0,1.2...| 44.80928824019599|
|30.1|[0.01709,90.0,2.0...|25.264400971073016|
|50.0|[0.02009,95.0,2.6...| 41.68916053418553|
|42.3|[0.02177,82.5,2.0...|36.146191514394395|
|16.5|[0.02498,0.0,1.89...|22.746885437097184|
|23.9|[0.02543,55.0,3.7...|27.769299940712642|
|30.8|[0.02763,75.0,2.9...| 30.72520875882616|
|25.0|[0.02875,28.0,15....|28.649351219272933|
|18.5|[0.03041,0.0,5.19...|19.578146685590635|
|34.9|[0.0315,95.0,1.47...|29.605091447377312|
|33.4|[0.03237,0.0,2.18...|28.946444601423302|
|19.5|[0.03427,0.0,5.19...| 20.31754513618022|
|19.4|[0.03466,35.0,6.0...|22.979032509331866|
|45.4|[0.03578,20.0,3.3...| 38.10797692996398|
|20.7|[0.0373